<a href="https://colab.research.google.com/github/DebenSalemme/llm-test-debs/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phi-3.5-mini-ITA fine-tuning notebook

In this notebook, we fine-tune Phi-3.5-mini-instruct on a good mix of English and Italian to improve performance in Italian.
We use [Spectrum](https://arxiv.org/abs/2406.06623) to selectively train the most informative layers of the model.

**👣 For a complete walk-through of the fine-tuning process, check out the [accompanying article](https://huggingface.co/blog/anakin87/spectrum).**


- [🪪 fine-tuned model: Phi-3.5-mini-ITA](https://huggingface.co/anakin87/Phi-3.5-mini-ITA)
- [💬🇮🇹 Chat with the model](https://huggingface.co/spaces/anakin87/Phi-3.5-mini-ITA)

## Setup

In [9]:
! pip install datasets transformers trl accelerate scipy
! pip install ninja packaging
! MAX_JOBS=6 pip install flash-attn --no-build-isolation --upgrade
! pip install wandb

## Data preparation

The datasets used have different formats.
We prepare and mix them in a single dataset.

In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer


# Load and process FineTome dataset
finetome_ds = load_dataset("mlabonne/FineTome-100k")["train"]
mapping_keys, mapping_values = {"from": "role", "value": "content"}, {"human": "user", "gpt": "assistant"}

def process_conversation(row):
    conv = row["conversations"]
    new_conv = [{mapping_keys[k]: mapping_values.get(v, v) for k, v in msg.items()} for msg in conv]
    return {"conversations": new_conv}

finetome_ds = Dataset.from_list([process_conversation(row) for row in finetome_ds])

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

def apply_template(examples):
    text = [tokenizer.apply_chat_template(msg, tokenize=False, add_generation_prompt=False) for msg in examples["conversations"]]
    return {"text": text}

finetome_ds = finetome_ds.map(apply_template, batched=True).remove_columns("conversations").shuffle(seed=42)
finetome_ds = finetome_ds.add_column("origin", ["finetome"] * len(finetome_ds))

# Load and process Capybara-Claude dataset
capyclaude_ds = load_dataset("efederici/capybara-claude-15k-ita", split="train")
capyclaude_ds = capyclaude_ds.map(apply_template, batched=True).remove_columns(["conversations", "hash"]).shuffle(seed=42)
capyclaude_ds = capyclaude_ds.add_column("origin", ["capyclaude"] * len(capyclaude_ds))

# Concatenate and split datasets
mixed_ds = concatenate_datasets([finetome_ds, capyclaude_ds]).shuffle(seed=42)
mixed_ds = mixed_ds.class_encode_column("origin").train_test_split(test_size=0.005, stratify_by_column="origin")


In [ ]:
mixed_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'origin'],
        num_rows: 114106
    })
    test: Dataset({
        features: ['text', 'origin'],
        num_rows: 574
    })
})

In [ ]:
# print(mixed_ds["train"][587]["text"])

We can then check how many examples will be truncated if we choose a maximum length of X tokens (2048 in this case).

In [ ]:
# from scipy.stats import percentileofscore
# import multiprocessing

# def calculate_lengths(batch):
#     return {"conv_lengths": [len(tokenizer(text)["input_ids"]) for text in batch["text"]]}

# conv_lengths = mixed_ds["train"].map(
#     calculate_lengths,
#     batched=True,
#     batch_size=1000,
#     num_proc=multiprocessing.cpu_count()
# )["conv_lengths"]

In [ ]:
# chosen_length=2048

# percentile = percentileofscore(conv_lengths, chosen_length)
# print(percentile)

## Load model

For Spectrum, we need to load the model using Transformers, no quantization.

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model_id = "microsoft/Phi-3.5-mini-instruct"


model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    use_cache=False,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
    trust_remote_code=True
)

# reference: https://huggingface.co/microsoft/Phi-3.5-mini-instruct/blob/main/sample_finetune.py
# keep in mind that setting tokenizer.model_max_length = 2048 as suggested is WRONG
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

## Apply Spectrum
https://github.com/cognitivecomputations/spectrum
https://arxiv.org/abs/2406.06623

In short, when using Spectrum, we only fine-tune some layers of the model with high Signal-to-Noise Ratio.
So, we need to freeze the other layers before training.

---

I computed the following YAML file using the Spectrum script, which unfortunately is not compatible with notebook environments.

```bash
# installation
git clone https://github.com/cognitivecomputations/spectrum.git
cd spectrum
pip install -r requirements.txt

# run
python spectrum.py --model-name microsoft/Phi-3.5-mini-instruct --top-percent 30
```

This command first scans the model (if not available) and then produces the YAML file with top SNR layers.

In [ ]:
# For simplicity, I'm pasting the YAML parameters here

yaml_parameters="""unfrozen_parameters:
- ^lm_head.weight$
- ^model.embed_tokens.weight$
# mlp.down_proj layers
- model.layers.2.mlp.down_proj
- model.layers.3.mlp.down_proj
- model.layers.1.mlp.down_proj
- model.layers.23.mlp.down_proj
- model.layers.4.mlp.down_proj
- model.layers.26.mlp.down_proj
- model.layers.25.mlp.down_proj
- model.layers.24.mlp.down_proj
- model.layers.28.mlp.down_proj
# mlp.gate_up_proj layers
- model.layers.31.mlp.gate_up_proj
- model.layers.4.mlp.gate_up_proj
- model.layers.3.mlp.gate_up_proj
- model.layers.5.mlp.gate_up_proj
- model.layers.6.mlp.gate_up_proj
- model.layers.2.mlp.gate_up_proj
- model.layers.30.mlp.gate_up_proj
- model.layers.9.mlp.gate_up_proj
- model.layers.28.mlp.gate_up_proj
# self_attn.o_proj layers
- model.layers.0.self_attn.o_proj
- model.layers.1.self_attn.o_proj
- model.layers.10.self_attn.o_proj
- model.layers.11.self_attn.o_proj
- model.layers.9.self_attn.o_proj
- model.layers.3.self_attn.o_proj
- model.layers.19.self_attn.o_proj
- model.layers.8.self_attn.o_proj
- model.layers.4.self_attn.o_proj
# self_attn.qkv_proj layers
- model.layers.23.self_attn.qkv_proj
- model.layers.24.self_attn.qkv_proj
- model.layers.22.self_attn.qkv_proj
- model.layers.26.self_attn.qkv_proj
- model.layers.27.self_attn.qkv_proj
- model.layers.25.self_attn.qkv_proj
- model.layers.28.self_attn.qkv_proj
- model.layers.29.self_attn.qkv_proj
- model.layers.31.self_attn.qkv_proj
"""

In [ ]:
unfrozen_parameters = []
for line in yaml_parameters.splitlines():
  if line.startswith("- "):
    unfrozen_parameters.append(line.split("- ")[1])

In [ ]:
import re

def _freeze_and_unfreeze_parameters(model, unfrozen_parameters):
    # freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    # unfreeze Spectrum parameters
    for name, param in model.named_parameters():
        if any(re.match(unfrozen_param, name) for unfrozen_param in unfrozen_parameters):
            param.requires_grad = True

In [ ]:
_freeze_and_unfreeze_parameters(model, unfrozen_parameters)

In [ ]:
# check the outcome of our freezing operation
for name, param in model.named_parameters():
    if param.requires_grad:
      print(name, param.requires_grad)

# model.embed_tokens.weight True
# model.layers.0.self_attn.o_proj.weight True
# model.layers.1.self_attn.o_proj.weight True
# model.layers.1.mlp.down_proj.weight True
# ...

## Training configuration

In [ ]:
# WANDB configuration (optional)

# import wandb
# run = wandb.init(...)

In [ ]:
from trl import SFTConfig, SFTTrainer

new_model_id="anakin87/Phi-3.5-mini-ITA"

cfg = SFTConfig(
    output_dir='./mymodel',
    overwrite_output_dir = True,
    hub_model_id=new_model_id,
    hub_strategy="every_save",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,
    push_to_hub=True,
    logging_steps=20,
    max_seq_length=2048,                    # see above in "Data preparation" section
    dataset_text_field="text",              # since we already prepared the dataset, let's point the Trainer to the correct column
    remove_unused_columns=True,
    packing=True,                           # speeds up training. https://huggingface.co/docs/trl/en/sft_trainer#packing-dataset--constantlengthdataset-
    num_train_epochs=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.2,
    bf16=True,
    tf32=True,
    learning_rate=5.0e-06,                  # suggested in https://huggingface.co/microsoft/Phi-3.5-mini-instruct/blob/main/sample_finetune.py
    per_device_train_batch_size=8,
)

In [ ]:
sft_trainer = SFTTrainer(
    model=model,
    args=cfg,
    train_dataset=mixed_ds["train"],

    tokenizer=tokenizer
)

In [ ]:
sft_trainer.train()

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
tokenizer.padding_side = 'left'

tokenizer.push_to_hub(new_model_id)

I finally did some manual updates on the model repo:
- copying some files from the original model to my model...
- modifying config.json and generation_config.json to use the right tokens ids for `eos_token_id`.